In [114]:
import numpy as np
import matplotlib.pyplot as plt
import math
import sympy as sp
from sympy.solvers import solve
import pandas as pd

### Модельная задача

### Аналитичское решение

In [115]:
x0 = 1/math.sqrt(5)
u0 = 1; u1 = 2

In [116]:
def analytic_solution(x0,u0,u1,x):
    from sympy.abc import a,b,c,d
    
    #expressions
    K = lambda x: math.sin(x**2)+1
    mju = lambda f,q: f/q
    q_a = lambda x: x
    q_b = lambda x: x**3
    f_a = 1
    f_b = lambda x: x**2 - 1
    lmb = lambda q,k: math.sqrt(q/k)
    u = lambda c1,c2,lmb,mju,x: c1*math.exp(lmb*x) + c2*math.exp(-lmb*x) + mju
    
    #constants
    k = K(x0)
    fa = f_a; fb = f_b(x0)
    qa = q_a(x0); qb = q_b(x0)
    lmba = lmb(qa,k); lmbb = lmb(qb,k)
    mjua = mju(fa,qa); mjub = mju(fb,qb)
    #equations
    eq = [a + b - u0 + mjua, a*math.exp(lmba*x0)+b*math.exp(-lmba*x0)-c*math.exp(lmbb*x0)-d*math.exp(-lmbb*x0)-mjub+mjua,
          k*lmba*a*math.exp(lmba*x0)-k*lmba*b*math.exp(-lmba*x0)-k*lmbb*c*math.exp(lmbb*x0)+k*lmbb*d*math.exp(-lmbb*x0),c*math.exp(lmbb)+d*math.exp(-lmbb)-u1+mjub]
    constants = solve(eq,[a,b,c,d],dict=False)
    c1 = constants[a]; c2 = constants[b]; c3 = constants[c]; c4 = constants[d]
    if x < x0:
        solution = u(c1,c2,lmba,mjua,x)
    if x > x0:
        solution = u(c3,c4,lmbb,mjub,x)
        
    return solution

In [117]:
def analytic_points(x0,u0,u1):
        x1 = np.linspace(0,1,11)
        y1 = []
        for x in x1:
                res1 = analytic_solution(x0,u0,u1,x)
                y1.append(res1)
        y1[0] = u0
        y1[-1] = u1
        return y1

In [118]:
a1 = analytic_points(x0,u0,u1)
print(a1)

[1, 1.09925868311849, 1.19427470611388, 1.28540267612549, 1.37298268977846, 1.45883994987694, 1.55125229030841, 1.65149670396797, 1.75964799622727, 1.87578687280662, 2]


### Метод прогонки

In [119]:
def progonka1(n,x0,u0,u1):
    h = 1/n
    x = np.linspace(0,1,n+1)
    setka1 = [i for i in x if i < x0]
    setka2 = [i for i in x if i > x0]
    du = lambda c1,c2,lmb,mju,x: c1*lmb*math.exp(lmb*x) + -lmb*c2*math.exp(-lmb*x) 
    #expressions
    K = lambda x: math.sin(x**2)+1
    mju = lambda f,q: f/q
    q_a = lambda x: x
    q_b = lambda x: x**3
    f_a = 1
    f_b = lambda x: x**2 - 1
    lmb = lambda q,k: math.sqrt(q/k)
    
    #constants
    k = K(x0)
    fa = f_a; fb = f_b(x0)
    qa = q_a(x0); qb = q_b(x0)
    lmba = lmb(qa,k); lmbb = lmb(qb,k)
    mjua = mju(fa,qa); mjub = mju(fb,qb)

    #progonga constants
    al = k ; bl = -2*k-qa*h**2; cl = k; dl = -fa*h**2
    aL = k; bL = -2*k-qb*h**2; cL = k; dL = -fb*h**2
    
    y1 = np.zeros(len(setka1)); y2 = np.zeros(len(setka2))
    y1[0] = u0; y2[-1] = u1
    alpha1 = np.zeros(len(setka1)); beta1 = np.zeros(len(setka1))
    alpha2 = np.zeros(len(setka2)); beta2 = np.zeros(len(setka2))
    alpha1[0] = -al/bl; alpha2[-1] = -cL/bL
    beta1[0] = (dl-cl*u0)/bl; beta2[-1] = (dL - cL*u1)/bL                                               
    
    for i in range(1,len(setka1)):
        alpha1[i] = -al/(bl+cl*alpha1[i-1])
        beta1[i] = (dl-cl*beta1[i-1])/(bl+cl*alpha1[i-1])
        
    for i in range(len(setka2)-2,-1,-1):
        alpha2[i] = -cL/(bL+aL*alpha2[i+1])
        beta2[i] = (dL-aL*beta2[i+1])/(bL+aL*alpha2[i+1])

    y1[-1] = (k*beta1[-2]+k*beta2[1])/(k*(1-alpha1[-2])+k*(1-alpha2[1]))
    y2[0] = y1[-1]

    #solutions
    solv1 = np.zeros(5); solv2 = np.zeros(6)
    solv2[-1] = u1; solv1[0] = u0; ind1 = 4; ind2 = 0
    xn1 = setka1[-1]; xn2 = setka2[0]; t2 = 0.5; t1 = 0.4
        
    for i in range(len(setka1)-2,-1,-1):
        y1[i] = alpha1[i]*y1[i+1]+beta1[i]
        if abs(xn1-t1) < 10**(-10):
            solv1[ind1] = y1[i]
            ind1 -= 1
            t1 -= 0.1
        xn1 -= h
    y1[0] = u0  
    for i in range(1,len(setka2)):
        y2[i] = alpha2[i]*y2[i-1] + beta2[i]
        if abs(xn2 - t2) < 10**(-10):
            solv2[ind2] = y2[i]
            ind2 += 1
            t2 += 0.1
        xn2 += h
    y2[-1] = u1

    solution = np.concatenate((solv1, solv2))
    y = np.concatenate((y1, y2))
    return solution
    

In [120]:
a1 = analytic_points(x0,u0,u1)
print(a1)

[1, 1.09925868311849, 1.19427470611388, 1.28540267612549, 1.37298268977846, 1.45883994987694, 1.55125229030841, 1.65149670396797, 1.75964799622727, 1.87578687280662, 2]


In [121]:
y = progonka1(10,x0,u0,u1); delta1 = []; x = np.linspace(0,1,11)
for i in range(len(y)):
    delta1.append(abs(y[i] - a1[i]))
    
df = pd.DataFrame({"x": x, "u мод. аналит.": a1, "u мод. числ. ": y ,"u мод. числ.- u мод. аналит. ": delta1})
display(df)
print("Максимальная норма ошибки модельной задачи =", max(delta1))

,x,u мод. аналит.,u мод. числ.,u мод. числ.- u мод. аналит.
0,0.0,1,1.000000,0.0
1,0.1,1.09925868311849,1.092023,0.00723528682919050
2,0.2,1.19427470611388,1.179778,0.0144962490756306
3,0.3,1.28540267612549,1.263593,0.0218100870180979
4,0.4,1.37298268977846,1.343778,0.0292041936995904
5,0.5,1.45883994987694,1.497492,0.0386522438994716
6,0.6,1.55125229030841,1.582141,0.0308882236562609
7,0.7,1.65149670396797,1.674643,0.0231467646982442
8,0.8,1.75964799622727,1.775070,0.0154220858256657
9,0.9,1.87578687280662,1.883495,0.00770841799290833


Максимальная норма ошибки модельной задачи = 0.0386522438994716


### Задача с переменными коеффициентами

In [122]:
def progonka2(n,x0,u0,u1):
    h = 1/n
    x = np.linspace(0,1,n+1)
    setka1 = [i for i in x if i < x0]
    setka2 = [i for i in x if i > x0]
    du = lambda c1,c2,lmb,mju,x: c1*lmb*math.exp(lmb*x) + -lmb*c2*math.exp(-lmb*x) 
    #expressions
    K = lambda x: math.sin(x**2)+1
    mju = lambda f,q: f/q
    q_a = lambda x: x
    q_b = lambda x: x**3
    f_a = 1
    f_b = lambda x: x**2 - 1
    lmb = lambda q,k: math.sqrt(q/k)
    
    #constants
    k = K(x0)
    fa = f_a; fb = f_b(x0)
    qa = q_a(x0); qb = q_b(x0)
    lmba = lmb(qa,k); lmbb = lmb(qb,k)
    mjua = mju(fa,qa); mjub = mju(fb,qb)

    #progonga constants
    al = lambda x: K(x+h/2); bl = lambda x: -K(x+h/2)-K(x-h/2)-q_a(x)*h**2; cl = lambda x: K(x-h/2); dl = lambda x: -f_a*h**2
    aL = lambda x: K(x+h/2); bL = lambda x: -K(x+h/2)-K(x-h/2)-q_b(x)*h**2; cL = lambda x: K(x-h/2); dL = lambda x: -f_b(x)*h**2
    
    y1 = np.zeros(len(setka1)); y2 = np.zeros(len(setka2))
    y1[0] = u0; y2[-1] = u1
    alpha1 = np.zeros(len(setka1)); beta1 = np.zeros(len(setka1))
    alpha2 = np.zeros(len(setka2)); beta2 = np.zeros(len(setka2))
    alpha1[0] = -al(0)/bl(0); alpha2[-1] = -cL(1)/bL(1)
    beta1[0] = (dl(0)-cl(0)*u0)/bl(0); beta2[-1] = (dL(1) - cL(1)*u1)/bL(1)                                               
    x1 = 0 + h ; x2 = 1 -h
    
    for i in range(1,len(setka1)):
        alpha1[i] = -al(x1)/(bl(x1)+cl(x1)*alpha1[i-1])
        beta1[i] = (dl(x1)-cl(x1)*beta1[i-1])/(bl(x1)+cl(x1)*alpha1[i-1])
        x1 += h
    for i in range(len(setka2)-2,-1,-1):
        alpha2[i] = -cL(x2)/(bL(x2)+aL(x2)*alpha2[i+1])
        beta2[i] = (dL(x2)-aL(x2)*beta2[i+1])/(bL(x2)+aL(x2)*alpha2[i+1])
        x2 -= h

    y1[-1] = (k*beta1[-2]+k*beta2[1])/(k*(1-alpha1[-2])+k*(1-alpha2[1]))
    y2[0] = y1[-1]

    #solutions
    solv1 = np.zeros(5); solv2 = np.zeros(6)
    solv2[-1] = u1; solv1[0] = u0; ind1 = 4; ind2 = 0
    xn1 = setka1[-1]; xn2 = setka2[0]; t2 = 0.5; t1 = 0.4
    
    for i in range(len(setka1)-2,-1,-1):
        y1[i] = alpha1[i]*y1[i+1]+beta1[i]
        if abs(xn1-t1) < 10**(-10):
            solv1[ind1] = y1[i]
            ind1 -= 1
            t1 -= 0.1
        xn1 -= h
    y1[0] = u0  
    for i in range(1,len(setka2)):
        y2[i] = alpha2[i]*y2[i-1] + beta2[i]
        if abs(xn2 - t2) < 10**(-10):
            solv2[ind2] = y2[i]
            ind2 += 1
            t2 += 0.1
        xn2 += h
    y2[-1] = u1

    solution = np.concatenate((solv1, solv2))
    y = np.concatenate((y1, y2))
    return solution

In [123]:
a2 = progonka2(10**6,x0,u0,u1)

In [124]:
u2 = progonka2(10,x0,u0,u1)
delta2 = []
for i in range(len(u2)):
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u аналит.": a2, "u числ. ": u2 ,"u числ.- u аналит. ": delta2})
display(df)
print("Максимальная норма ошибки =", max(delta2))

,x,u аналит.,u числ.,u числ.- u аналит.
0,0.0,1.000000,1.000000,0.000000
1,0.1,1.131919,1.109183,0.022736
2,0.2,1.252633,1.208390,0.044242
3,0.3,1.361796,1.297060,0.064736
4,0.4,1.459913,1.375423,0.084490
5,0.5,1.549539,1.513340,0.036199
6,0.6,1.638654,1.583671,0.054983
7,0.7,1.727778,1.655218,0.072560
8,0.8,1.816984,1.728521,0.088463
9,0.9,1.907149,1.804873,0.102276


Максимальная норма ошибки = 0.1022764293447942


## Таблица результатов

In [125]:
x = np.linspace(0,1,11)
a1 = analytic_points(x0,u0,u1)
a2 = progonka2(10**6,x0,u0,u1)

In [128]:
delta1 = []; delta2 = []
y1 = progonka1(20,x0,u0,u1)
y2 = progonka2(20,x0,u0,u1)
for i in range(len(y1)):
    delta1.append(abs(y1[i] - a1[i]))
    delta2.append(abs(y2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u мод. аналит.": a1, "u мод. числ. ": y1 ,"u мод. числ.- u мод. аналит. ": delta1, 
                  "u пер. аналит.": a2, "u пер. числ. ": y2 ,"u пер. числ.- u пер. аналит. ": delta2})
display(df)
print("Максимальная норма ошибки модельной задачи =", max(delta1))
print("Максимальная норма ошибки реальной задачи = ", max(delta2))

,x,u мод. аналит.,u мод. числ.,u мод. числ.- u мод. аналит.,u пер. аналит.,u пер. числ.,u пер. числ.- u пер. аналит.
0,0.0,1,1.000000,0.0,1.000000,1.000000,0.000000
1,0.1,1.09925868311849,1.094056,0.00520247490720194,1.131919,1.118329,0.013589
2,0.2,1.19427470611388,1.183851,0.0104240345106810,1.252633,1.226174,0.026459
3,0.3,1.28540267612549,1.269718,0.0156841921271944,1.361796,1.323092,0.038704
4,0.4,1.37298268977846,1.351980,0.0210026039849889,1.459913,1.409459,0.050453
5,0.5,1.45883994987694,1.473540,0.0147004303112401,1.549539,1.528379,0.021161
6,0.6,1.55125229030841,1.563000,0.0117474476185282,1.638654,1.608681,0.029973
7,0.7,1.65149670396797,1.660300,0.00880310932741568,1.727778,1.689487,0.038291
8,0.8,1.75964799622727,1.765513,0.00586521715607269,1.816984,1.771133,0.045851
9,0.9,1.87578687280662,1.878718,0.00293157754115336,1.907149,1.854726,0.052423


Максимальная норма ошибки модельной задачи = 0.0210026039849889
Максимальная норма ошибки реальной задачи =  0.05242280547707323
